# Predict the state-wide true case counts

This project uses the testing rates per state (from [https://covidtracking.com/](https://covidtracking.com/)), reported case counts by state and the reported mortality rates by state to **estimate the number of unreported (untested) COVID-19 cases in each U.S. state.**

The analysis makes a few assumptions:

1. The probability that a case is reported by a state is a function of the number of tests run per person in that state.  Hence the degree of under-reported cases is a function of tests run per capita.
2. The underlying mortality rate is the same across every state.
3. Patients take time to succumb to COVID-19, so the mortality counts *today* reflect the case counts *7 days ago*.  E.g., mortality rate = (cumulative deaths today) / (cumulative cases 7 days ago).

In all, the model attempts to find the most likely relationship between state-wise test volume (per capita) and under-reporting, such that the true underlying mortality rates between the individual states are as similar as possible.  The model simultaneously finds the most likely posterior distribution of mortality rates, the most likely *true* case count per state, and the test volume vs. case underreporting relationship.

Please see the source code in [source](source).

In [ ]:
# Setup and imports
%matplotlib inline

import warnings
warnings.simplefilter('ignore')

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import pymc3 as pm

from source import data, model

In [ ]:
df, df_pred = data.get_statewise_testing_data()

In [ ]:
print(df.shape)
print(df_pred.shape)
print(set(df_pred.index) - set(df.index))

In [ ]:
# Initialize the model:
# import importlib
# importlib.reload(model)
mod = model.case_count_model_us_states(df)

In [ ]:
# Run MCMC sampler
with mod:
    trace = pm.sample(500, tune=500, chains=1)

In [ ]:
# Visualize the trace of the MCMC sampler to assess convergence
ax = pm.traceplot(trace)

In [ ]:
n = len(trace['beta'])

# South Korea:
ppt_sk = np.log10(51500000. / 250000)

# Plot pop/test vs. Prob of case detection for all posterior samples:
x = np.linspace(0.0, 4.0, 101)
logit_pcase = pd.DataFrame([
    trace['alpha'][i] + trace['beta'][i] * x
    for i in range(n)])
pcase = np.exp(logit_pcase) / (np.exp(logit_pcase) + 1)

fig, ax = plt.subplots(1, 1, figsize=(12, 8))
for i in range(n):
    ax = plt.plot(10**(ppt_sk + x), pcase.iloc[i], color='grey', lw=.1, alpha=.5)
    plt.xscale('log')
    plt.xlabel('State-wise population per test', size=14)
    plt.ylabel('Probability a true case is detected', size=14)

# rug plots:
ax=plt.plot(df_pred['people_per_test'], np.zeros(len(df_pred)),
            marker='|', color='k', ls='', ms=20,
           label='U.S. State-wise Test Capacity Now')
ax=plt.plot(df['people_per_test_7_days_ago'], np.zeros(len(df)),
            marker='+', color='c', ls='', ms=10,
           label='U.S. State-wise Test Capacity 7 Days Ago')
ax = plt.legend(fontsize='x-large')

In [ ]:
n = len(trace['beta'])

# South Korea:
ppt_sk = np.log10(51500000. / 250000)


# Compute predicted case counts per state right now
logit_now = pd.DataFrame([
    pd.Series(np.random.normal((trace['alpha'][i] + trace['beta'][i] * (np.log10(df_pred['people_per_test']) - ppt_sk)),
                     trace['sigma'][i]), index=df_pred.index)
    for i in range(len(trace['beta']))])
prob_missing_now = np.exp(logit_now) / (np.exp(logit_now) + 1) 

predicted_counts_now = np.round(df_pred['positive'] / prob_missing_now.mean(axis=0)).astype(int)

predicted_counts_now_lower = np.round(df_pred['positive'] / prob_missing_now.quantile(0.975, axis=0)).astype(int)
predicted_counts_now_upper = np.round(df_pred['positive'] / prob_missing_now.quantile(0.025, axis=0)).astype(int)

case_increase_percent = list(map(lambda x, y: (((x - y) / float(y))),
                                 predicted_counts_now, df_pred['positive']))

df_summary = pd.DataFrame(
    data = {
     'Cases Reported': df_pred['positive'],
     'Cases Estimated': predicted_counts_now,
     'Percent Increase': case_increase_percent,
     'Tests per Million People': df_pred['tests_per_million'].round(1),
     'Cases Estimated (range)': list(map(lambda x, y: '(%i, %i)' % (round(x), round(y)),
                                        predicted_counts_now_lower, predicted_counts_now_upper))
    },
    index=df_pred.index)
df_summary.sort_values(by='Cases Estimated', ascending=False).style.background_gradient(
    cmap='Reds').format({'Percent Increase': "{:.1%}"})

In [ ]:
from datetime import datetime
print("Total for the United States on %s:" % str(datetime.today())[:10])
print("Reported Case Count: %i" % df_summary['Cases Reported'].sum())
print("Predicted Case Count: %i" % df_summary['Cases Estimated'].sum())
case_increase_percent = 100. * (df_summary['Cases Estimated'].sum() - df_summary['Cases Reported'].sum()) / df_summary['Cases Estimated'].sum()
print("Percentage Underreporting in Case Count: %.1f%%" % case_increase_percent)

In [ ]:
#collapse-hide

xerr = [df_summary['Cases Estimated'] - predicted_counts_now_lower, predicted_counts_now_upper - df_summary['Cases Estimated']]

fig, axs = plt.subplots(1, 1, figsize=(15, 15))
ax = plt.errorbar(df_summary['Cases Estimated'], range(len(df_summary)-1, -1, -1), xerr=xerr,
                  fmt='o', elinewidth=1, label='Estimate')
ax = plt.yticks(range(len(df_summary)), df_summary.index[::-1])
ax = plt.errorbar(df_summary['Cases Reported'], range(len(df_summary)-1, -1, -1), xerr=None,
                  fmt='.', color='k', label='Observed')
ax = plt.xlabel('COVID-19 Case Counts', size=20)
ax = plt.legend(fontsize='xx-large')
ax = plt.grid(linestyle='--', color='grey', axis='x')

In [ ]:
# Posterior plot for mu0
ax = pm.plot_posterior(trace, var_names=['mu_0'], figsize=(14, 6), textsize=18,
                       credible_interval=0.95, bw=5.0, lw=3, kind='kde')
ax = plt.xlabel('Overall true COVID-19 mortality rate', size=20)